In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [18]:
def eta(lamb):
    """computes eta(lamb) = -lamb"""
    return -lamb

def eta_inv(eta):
    """recovers lamb"""
    return -eta

def eta_inv_jac(eta):
    """compute the jacobian of eta^{-1}(eta)"""
    return -1


In [46]:
#Class computing the posterior of \eta(\theta) given observations

class Exponential():
    def __init__(self, eta_prior_mean, eta_prior_cov):
        self.eta_prior_mean = eta_prior_mean #prior on eta
        self.eta_prior_cov = eta_prior_cov #prior on cov of eta
        self.num_params = 1

        self.dr_ = lambda x : 1.
        self.db_ = lambda x : 0.
        self.ddr_ = lambda x : 0.

        self.m = lambda x : x
        self.dmmdr = lambda x : 2 * x

        return
    
    def posterior(self, data):
        assert data.ndim == 1, "Data must be 1-dimensional"

        Lambda = 0.
        Nu = 0.

        T = len(data)

        for x in data:
            Lambda += (1/T) * self.dr_(x) * self.m(x)**2 * self.dr_(x)
            Nu += (2/T) * (self.dr_(x) * self.db_(x) + self.dmmdr(x))

        eta_post_cov = ((self.eta_prior_cov)**(-1) + 2 * T * Lambda)**(-1)

        eta_post_mean = eta_post_cov * ((self.eta_prior_cov)**(-1) * self.eta_prior_mean - T * Nu)


        return eta_post_mean, eta_post_cov

In [103]:
true_lamb = 1.4
size = 10000

data = np.random.exponential(scale=1/true_lamb, size=size)

#Setting the prior on theta = lambda
theta_prior_mean = 2.

#converting the prior mu and sigma2 to natural parameters
eta_prior_mean = eta(theta_prior_mean)
eta_prior_cov = 1.0

#instantiating the LogNormal class and obtaining the posterior mean and covariance for the r.v. \eta | data 
exp_wrong = Exponential(eta_prior_mean=eta_prior_mean, eta_prior_cov=eta_prior_cov)
eta_post_mean, eta_post_cov = exp_wrong.posterior(data=data)

#converting back from eta to mu, sigma2, and obtaining the mean and covariance for mu and sigma2
theta_post_mean = eta_inv(eta_post_mean)
theta_post_cov = eta_inv_jac(eta_post_mean) * eta_post_cov * eta_inv_jac(eta_post_mean)

print("Frequentist estimate  : ", (1/data.mean()))
print("theta from Exp-Normal : ", theta_post_mean, "+-", 2*np.sqrt(theta_post_cov))
print("theta (true)          : ", true_lamb)


Frequentist estimate  :  1.3824690974395373
theta from Exp-Normal :  1.3976566411501072 +- 0.013899940873723857
theta (true)          :  1.4
